In [1]:
import sys
import numpy as np
import scipy.stats as stats
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio
import math

init_notebook_mode(connected=True)


def process_clock(line):
	time = float(line.split()[0].replace(',','.'))
	#print('time ',time)
	return time

def process_icp(line):
	icp = float(line.split('#')[1].split()[0].replace(',','.'))
	#print('icp',icp)
	return icp

def process_branch(line):
	branch_misses =  int(line.split('#')[0].split()[0].replace('.',''))
	#print(branch_misses)
	return branch_misses

def process_cache(line):
	cache_misses = int(line.split('#')[0].split()[0].replace('.','') )
	#print('cache_misses', cache_misses)
	return cache_misses

def collect_data():
	file_name = sys.argv[1]
	plats = ['i5', 'rasp']
	tests = ['int_mat','float_mat','float_gauss','dep','branch','branch_sort']
	

	results = dict()
	for plat in plats:
		for test in tests:
			test_name = '{}_{}'.format(test,plat)
			file_name = 'result_{}.txt'.format(test_name)

			content = []
			with open(file_name,'r') as f:
				content = f.readlines()
				#print(content)

			content = [x.strip() for x in content]
			#print(content)

			test_num = -1
			results[test_name] = []
			for line in content:
				if('task-clock' in line):
					test_num += 1
					results[test_name].append(dict())
					results[test_name][test_num]['clock'] = process_clock(line)
				if('instructions' in line):
					results[test_name][test_num]['icp'] = process_icp(line)
                    
				if('branch-misses' in line):
					results[test_name][test_num]['branch_misses'] = process_branch(line)
                    
				if('L1-dcache-load-misses' in line):
					results[test_name][test_num]['cache_misses'] = process_cache(line)
			#print('resultado do teste {}'.format(test_name))
			#print(results[test_name])
	return results


test_results = collect_data()
test_results

{'int_mat_i5': [{'clock': 5541.534155,
   'icp': 2.61,
   'branch_misses': 1040593,
   'cache_misses': 1202622113},
  {'clock': 5558.865999,
   'icp': 2.6,
   'branch_misses': 1042062,
   'cache_misses': 1203039621},
  {'clock': 5587.592536,
   'icp': 2.58,
   'branch_misses': 1039881,
   'cache_misses': 1204618275},
  {'clock': 5568.836711,
   'icp': 2.61,
   'branch_misses': 1041086,
   'cache_misses': 1209492142},
  {'clock': 5483.359369,
   'icp': 2.63,
   'branch_misses': 1041551,
   'cache_misses': 1197951203},
  {'clock': 5556.287346,
   'icp': 2.61,
   'branch_misses': 1040512,
   'cache_misses': 1209664055},
  {'clock': 5476.921236,
   'icp': 2.62,
   'branch_misses': 1040962,
   'cache_misses': 1202598640},
  {'clock': 5515.66087,
   'icp': 2.62,
   'branch_misses': 1041032,
   'cache_misses': 1204586470},
  {'clock': 5504.107633,
   'icp': 2.62,
   'branch_misses': 1040538,
   'cache_misses': 1207184566},
  {'clock': 5514.882283,
   'icp': 2.63,
   'branch_misses': 1041511,


In [2]:
def confidence(avg, std, size):
    return 1.96*std/math.sqrt(size)

def plot_tables():
    avgs = dict()
    stds = dict()
    confs = dict()

    for test, data in test_results.items():
        clock_data = np.array([])
        icp_data = np.array([])
        branch_data = np.array([])
        cache_data = np.array([])

        #print('teste: {}'.format(test))
        for results in test_results[test]:
            clock_data = np.append(clock_data, results['clock'])
            icp_data = np.append(icp_data, results['icp'])
            branch_data = np.append(branch_data, results['branch_misses'])
            cache_data = np.append(cache_data, results['cache_misses'])
            #print(results)
        #print('clock data', clock_data)
        clock_avg = round(np.average(clock_data),3)
        clock_std = round(np.std(clock_data),3)
        clock_conf = round(confidence(clock_avg, clock_std, 10),3)
        #print('clock avg {} clock std {} confidence {}\n'.format(clock_avg, clock_std, clock_conf))
        clock_data = np.append(clock_data, clock_avg)
        clock_data = np.append(clock_data, clock_std)
        clock_data = np.append(clock_data, clock_conf)


        #print('icp data ', icp_data)
        icp_avg = round(np.average(icp_data),3)
        icp_std = round(np.std(icp_data),3)
        icp_conf = round(confidence(icp_avg, icp_std, 10),3)
        #print('icp avg {} icp std {} icp conf {} \n'.format(icp_avg, icp_std, icp_conf))
        icp_data = np.append(icp_data, icp_avg)
        icp_data = np.append(icp_data, icp_std)
        icp_data = np.append(icp_data, icp_conf)

        #print('branch data', branch_data)
        branch_avg = round(np.average(branch_data),3)
        branch_std = round(np.std(branch_data),3)
        branch_conf = round(confidence(branch_avg, branch_std, 10),3)
        print('branch conf do teste {}: {}'.format(test, branch_conf))
        #print('branch avg {} branch std {} branch conf {}\n'.format(branch_avg, branch_std, branch_conf))
        branch_data = np.append(branch_data, branch_avg)
        branch_data = np.append(branch_data, branch_std)
        branch_data = np.append(branch_data, branch_conf)

        #print('cache data', cache_data)
        cache_avg = round(np.average(cache_data), 3)
        cache_std = round(np.std(cache_data), 3)
        cache_conf = round(confidence(cache_avg, cache_std, 10), 3)
        #print('cache avg {} cache std {} cache conf {}\n'.format(cache_avg, cache_std, cache_conf))
        cache_data = np.append(cache_data, cache_avg)
        cache_data = np.append(cache_data, cache_std)
        cache_data = np.append(cache_data, cache_conf)

        index_cells = [i for i in range(1,11)]
        index_cells.append('Media')
        index_cells.append('Desvio padrao')
        index_cells.append('Intervalo de conf.(95%)+-')
        cells = [index_cells,clock_data, icp_data, branch_data, cache_data]
        columns = ['Amostra','Tempo(ms)','IPC', 'Branch misses', 'Cache misses']

        layout = go.Layout(
            title=test,
            font=dict(family='Courier New, monospace', size=15, color='#7f7f7f'),
            width=1000,
            height=800
        )
        trace = go.Table(
            columnwidth = [40],
            header=dict(values=columns),
            cells=dict(values=cells, height=40)
        )
        data = [trace]
        figure = go.Figure(layout=layout, data=data)
        iplot(figure)

        avgs[test] = dict()
        avgs[test]['clock'] = clock_avg
        avgs[test]['ipc'] = icp_avg
        avgs[test]['branch'] = branch_avg
        avgs[test]['cache'] = cache_avg

        confs[test] = dict()
        confs[test]['clock'] = clock_conf
        confs[test]['ipc'] = icp_conf
        confs[test]['branch'] = branch_conf
        confs[test]['cache'] = cache_conf
        #print('------------------')
        pio.write_image(figure, 'images/{}.jpeg'.format(test))
        #print(avgs)
    return avgs, confs



avgs, confs = plot_tables()

branch conf do teste int_mat_i5: 370.042


branch conf do teste float_mat_i5: 1129.298


branch conf do teste float_gauss_i5: 52862.389


branch conf do teste dep_i5: 3464.095


branch conf do teste branch_i5: 244.026


...

branch conf do teste float_mat_rasp: 831247.14


branch conf do teste float_gauss_rasp: 122986.831


branch conf do teste dep_rasp: 71522.189


branch conf do teste branch_rasp: 124.774


branch conf do teste branch_sort_rasp: 3818.879


In [3]:
plot_procs = 'Intel Core i5 vs ARM Cortex A7'
labels = []
labels = ['Core i5', 'ARM']

clock = [avgs['int_mat_i5']['clock'], avgs['int_mat_rasp']['clock']]
icp = [avgs['int_mat_i5']['ipc'], avgs['int_mat_rasp']['ipc']]
branch = [avgs['int_mat_i5']['branch'], avgs['int_mat_rasp']['branch']]
cache = [avgs['int_mat_i5']['cache'], avgs['int_mat_rasp']['cache']]

clock_confs = [confs['int_mat_i5']['clock'], confs['int_mat_rasp']['clock']]
print(clock_confs)

icp_confs = [confs['int_mat_i5']['ipc'], confs['int_mat_rasp']['ipc']]
print(icp_confs)

cache_confs = [confs['int_mat_i5']['cache'], confs['int_mat_rasp']['cache']]
print(cache_confs)

branch_confs = [confs['int_mat_i5']['branch'], confs['int_mat_rasp']['branch']]
print(branch_confs)

def plot_bar_data(data, confs, labels, title):
    trace = go.Scatter(
        x = labels,
        y = data,
        mode='markers',
        error_y = dict(
            type='data',
            array=confs,
            visible=True
        )
        #marker=dict(color=['#1F77B4', '#30c95e'])
    )
    layout = go.Layout(
        title=title
    )
    data = [trace]
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
    pio.write_image(fig, 'images/{}.jpeg'.format(title))

    return 0

plot_bar_data(clock, clock_confs,labels, '{} - Tempo (Multiplicação de matriz de inteiros)'.format(plot_procs))
plot_bar_data(icp, icp_confs, labels, '{} - IPC (Multiplicação de matriz de inteiros)'.format(plot_procs))
plot_bar_data(branch, branch_confs, labels, '{} - Branch misses (Multiplicação de matriz de inteiros)'.format(plot_procs))
plot_bar_data(cache, cache_confs, labels, '{} - Cache misses (Multiplicação de matriz de inteiros)'.format(plot_procs))


[21.978, 602.411]
[0.009, 0.003]
[2064401.332, 2905905.715]
[370.042, 417997.755]


0

In [4]:
print(avgs['float_mat_i5'])
print(avgs['float_mat_rasp'])

clock = [avgs['float_mat_i5']['clock'], avgs['float_mat_rasp']['clock']]
icp = [avgs['float_mat_i5']['ipc'], avgs['float_mat_rasp']['ipc']]
branch = [avgs['float_mat_i5']['branch'], avgs['float_mat_rasp']['branch']]
cache = [avgs['float_mat_i5']['cache'], avgs['float_mat_rasp']['cache']]

clock_confs = [confs['float_mat_i5']['clock'], confs['float_mat_rasp']['clock']]
print(clock_confs)

icp_confs = [confs['float_mat_i5']['ipc'], confs['float_mat_rasp']['ipc']]
print(icp_confs)

cache_confs = [confs['float_mat_i5']['cache'], confs['float_mat_rasp']['cache']]
print(cache_confs)

branch_confs = [confs['float_mat_i5']['branch'], confs['float_mat_rasp']['branch']]
print(branch_confs)

plot_bar_data(clock, clock_confs, labels, '{} - Tempo (Multiplicação de matriz de floats)'.format(plot_procs))
plot_bar_data(icp, icp_confs, labels, '{} - IPC (Multiplicação de matriz de floats)'.format(plot_procs))
plot_bar_data(branch, branch_confs, labels, '{} - Branch misses (Multiplicação de matriz de floats)'.format(plot_procs))
plot_bar_data(cache, cache_confs, labels, '{} - Cache misses (Multiplicação de matriz de floats)'.format(plot_procs))

{'clock': 7485.323, 'ipc': 1.924, 'branch': 1057191.7, 'cache': 1388882447.0}
{'clock': 110346.756, 'ipc': 0.345, 'branch': 2721164.0, 'cache': 1480177959.1}
[109.631, 1312.622]
[0.029, 0.003]
[12273535.985, 3915943.072]
[1129.298, 831247.14]


0

In [5]:
print(avgs)

clock = [avgs['float_gauss_i5']['clock'], avgs['float_gauss_rasp']['clock']]
icp = [avgs['float_gauss_i5']['ipc'], avgs['float_gauss_rasp']['ipc']]
branch = [avgs['float_gauss_i5']['branch'], avgs['float_gauss_rasp']['branch']]
cache = [avgs['float_gauss_i5']['cache'], avgs['float_gauss_rasp']['cache']]

clock_confs = [confs['float_gauss_i5']['clock'], confs['float_gauss_rasp']['clock']]
print(clock_confs)

icp_confs = [confs['float_gauss_i5']['ipc'], confs['float_gauss_rasp']['ipc']]
print(icp_confs)

cache_confs = [confs['float_gauss_i5']['cache'], confs['float_gauss_rasp']['cache']]
print(cache_confs)

branch_confs = [confs['float_gauss_i5']['branch'], confs['float_gauss_rasp']['branch']]
print(branch_confs)

plot_bar_data(clock, clock_confs, labels, '{} - Tempo (Escalonamento de Gauss)'.format(plot_procs))
plot_bar_data(icp, icp_confs, labels, '{} - IPC (Escalonamento de Gauss)'.format(plot_procs))
plot_bar_data(branch, branch_confs, labels, '{} - Branch misses (Escalonamento de Gauss)'.format(plot_procs))
plot_bar_data(cache, cache_confs, labels, '{} - Cache misses (Escalonamento de Gauss)'.format(plot_procs))

{'int_mat_i5': {'clock': 5530.805, 'ipc': 2.613, 'branch': 1040972.8, 'cache': 1204601072.2}, 'float_mat_i5': {'clock': 7485.323, 'ipc': 1.924, 'branch': 1057191.7, 'cache': 1388882447.0}, 'float_gauss_i5': {'clock': 3714.823, 'ipc': 1.748, 'branch': 4099589.8, 'cache': 229916209.4}, 'dep_i5': {'clock': 813.061, 'ipc': 1.295, 'branch': 85191.6, 'cache': 17780976.3}, 'branch_i5': {'clock': 0.76, 'ipc': 1.321, 'branch': 22019.5, 'cache': 15123.8}, 'branch_sort_i5': {'clock': 9.925, 'ipc': 1.996, 'branch': 160088.8, 'cache': 24916.8}, 'int_mat_rasp': {'clock': 95407.217, 'ipc': 0.394, 'branch': 4037964.6, 'cache': 1231935837.8}, 'float_mat_rasp': {'clock': 110346.756, 'ipc': 0.345, 'branch': 2721164.0, 'cache': 1480177959.1}, 'float_gauss_rasp': {'clock': 7011.943, 'ipc': 0.52, 'branch': 3731672.4, 'cache': 16136505.5}, 'dep_rasp': {'clock': 8644.85, 'ipc': 0.88, 'branch': 208778.2, 'cache': 2556415.0}, 'branch_rasp': {'clock': 5.893, 'ipc': 1.788, 'branch': 26216.3, 'cache': 6285.0}, 'br

0

In [9]:
#print(avgs)
clock = [avgs['branch_i5']['clock'], avgs['branch_rasp']['clock']]
clock_confs = [confs['branch_i5']['clock'], confs['branch_rasp']['clock']]
icp = [avgs['branch_i5']['ipc'], avgs['branch_rasp']['ipc']]
icp_confs = [confs['branch_i5']['ipc'], confs['branch_rasp']['ipc']]
branch = [avgs['branch_i5']['branch'], avgs['branch_rasp']['branch']]
branch_confs = [confs['branch_i5']['branch'], confs['branch_rasp']['branch']]
cache = [avgs['branch_i5']['cache'], avgs['branch_rasp']['cache']]
cache_confs = [confs['branch_i5']['cache'], confs['branch_rasp']['cache']]

clock_sorted = [avgs['branch_sort_i5']['clock'], avgs['branch_sort_rasp']['clock']]
clock_sorted_confs = [confs['branch_sort_i5']['clock'], confs['branch_sort_rasp']['clock']]
icp_sorted = [avgs['branch_sort_i5']['ipc'], avgs['branch_sort_rasp']['ipc']]
icp_sorted_confs = [confs['branch_sort_i5']['ipc'], confs['branch_sort_rasp']['ipc']]
branch_sorted = [avgs['branch_sort_i5']['branch'], avgs['branch_sort_rasp']['branch']]
branch_sorted_confs = [confs['branch_sort_i5']['branch'], confs['branch_sort_rasp']['branch']]
cache_sorted = [avgs['branch_sort_i5']['cache'], avgs['branch_sort_rasp']['cache']]
cache_sorted_confs = [confs['branch_sort_i5']['cache'], confs['branch_sort_rasp']['cache']]

clock_data = clock + clock_sorted
print(clock)
print(clock_sorted)
print('clock data', clock_data)
clock_comb_conf = clock_confs + clock_sorted_confs
icp_data = icp + icp_sorted
icp_comb_conf = icp_confs + icp_sorted_confs
branch_data = branch + branch_sorted
branch_comb_conf = branch_confs + branch_sorted_confs
cache_data = cache + cache_sorted
cache_comb_conf = cache_confs + cache_sorted_confs

labels = ['i5 (não ordenado)', 'ARM (não ordenado)', 'i5 (ordenado)', 'ARM (ordenado)']
plot_bar_data(clock_data, clock_comb_conf, labels, '{} - Tempo (Dependencia de controle)'.format(plot_procs))
plot_bar_data(icp_data, icp_comb_conf, labels, '{} - IPC (Dependencia de controle)'.format(plot_procs))
plot_bar_data(branch_data, branch_comb_conf, labels, '{} - Branch misses (Dependencia de controle)'.format(plot_procs))
plot_bar_data(cache_data, cache_comb_conf, labels, '{} - Cache misses (Dependencia de controle)'.format(plot_procs))

[0.76, 5.893]
[9.925, 82.202]
clock data [0.76, 5.893, 9.925, 82.202]


0

In [7]:
clock = [avgs['dep_i5']['clock'], avgs['dep_rasp']['clock']]
icp = [avgs['dep_i5']['ipc'], avgs['dep_rasp']['ipc']]
branch = [avgs['dep_i5']['branch'], avgs['dep_rasp']['branch']]
cache = [avgs['dep_i5']['cache'], avgs['dep_rasp']['cache']]

clock_confs = [confs['dep_i5']['clock'], confs['dep_rasp']['clock']]
print(clock_confs)

icp_confs = [confs['dep_i5']['ipc'], confs['dep_rasp']['ipc']]
print(icp_confs)

cache_confs = [confs['dep_i5']['cache'], confs['dep_rasp']['cache']]
print(cache_confs)

branch_confs = [confs['dep_i5']['branch'], confs['dep_rasp']['branch']]
print(branch_confs)


plot_bar_data(clock, clock_confs, labels, '{} - Tempo (Dependência de dados)'.format(plot_procs))
plot_bar_data(icp, icp_confs, labels, '{} - IPC (Dependência de dados)'.format(plot_procs))
plot_bar_data(branch, branch_confs, labels, '{} - Branch misses (Dependência de dados)'.format(plot_procs))
plot_bar_data(cache, cache_confs, labels, '{} - Cache misses (Dependência de dados)'.format(plot_procs))

[4.274, 15.392]
[0.003, 0.0]
[248283.028, 444522.893]
[3464.095, 71522.189]


0